## Copy optimam cases, depending on sub-type classification 

Gets optimam cases into folder separated by System (HOLOGIC), FFDM (full image), ROI (lesion roi), norm_roi (normal random roi).

In [ ]:
import omidb
import numpy as np
import os
from pydicom import dcmread
from pydicom.pixel_data_handlers.util import apply_voi_lut
from  matplotlib import pyplot as plot

import cv2
import csv


In [ ]:
reading_path = '/media/robert/ICEBERG_1/img/optimam/image_db'
output_path = '/media/robert/ICEBERG_1/img/optimam/iceberg_selection'


In [ ]:
#episode_type = {SCREENING,BIOPSYWIDE,ASSESSMENT }
system_type = ['HOLOGIC', 'SIEMENS', 'GE']
#classification_type = 'NORMAL'}
debug = True
ER = 0
PR = 1
HER2 = 2
normal_roi_size = 150 # size of a normal ROI
normal_roi_noise = 500 #random noise to obtain the normal ROI from the center. 
extra_size = 50 # adding 50 pixels around the lesion

In [ ]:
# clients = ['demd2', 'demd3']
# clients = ['demd100018', 'demd128247', 'demd843','demd94678'] # second test. 
# db = omidb.DB(reading_path, clients= clients)

db = omidb.DB(reading_path) # all clients



In [ ]:
class stats:
    def __init__(self,N, M, IC):
        self.N = N
        self.M = M
        self.IC = IC
        self.image_CC = 0
        self.image_MLO = 0
        self.image_R = 0
        self.image_L = 0
        self.subtype = np.zeros(8, dtype=np.int32)

    def __repr__(self):
        return "Stats [N: %i, M %i, IC %i, CC: %i MLO:%i R: %i L:%i Subtype:%s ]"\
    % (self.N,self.M, self.IC,self.image_CC,self.image_MLO,self.image_R,self.image_L,np.array2string(self.subtype))
    

## Get random bounding box on center of mammogram.



In [ ]:
def is_overlapping1D(bbox1, bbox2):
    return bbox1[1] >= bbox2[0] and bbox2[1] >= bbox1[0]

def is_overlapping2D(bbox1, bbox2):
   
    return is_overlapping1D([bbox1.x1, bbox1.x2], [bbox2.x1, bbox2.x2]) \
       and is_overlapping1D([bbox1.y1, bbox1.y2], [bbox2.y1, bbox2.y2]) 

In [ ]:
def get_random_bbox(bbox, bbox_roi,mask):
    dims = mask.shape
    center_x =np.round((bbox.x2 - bbox.x1)/2).astype(int) # center of the image
    center_y =np.round((bbox.y2 - bbox.y1)/2).astype(int) # center of the image
    idx = 0
    found = False
    while not found and idx < 100 : 
        x = center_x + np.random.randint(-normal_roi_noise, +normal_roi_noise)
        y = center_y + np.random.randint(-normal_roi_noise, +normal_roi_noise)
        y1 = np.maximum(y- normal_roi_size,0)
        y2 = np.minimum(y+ normal_roi_size,dims[0])
        x1 = np.maximum(x- normal_roi_size,0)
        x2 = np.minimum(x+ normal_roi_size,dims[1])
        bbox_random = omidb.mark.BoundingBox(x1,y1,x2,y2)
        mask2 = mask.copy()
        mask2[:] = 0
        mask2[bbox_random.y1 : bbox_random.y2,bbox_random.x1:bbox_random.x2] = 255
        mask2[bbox_roi.y1 : bbox_roi.y2,bbox_roi.x1:bbox_roi.x2] = 128       
        
        # print("roi bbox ", bbox_roi)
        # print("random bbox ", bbox_random)
#         plot.imshow(mask2,'gray')
#         plot.show()
#         plot.imshow(mask,'gray')
#         plot.show()
        found = not (is_overlapping2D (bbox_random,bbox_roi) or x1 ==0 or y1 ==0)
        # apply bitwise on the inverse background (mask) and check all is inside the breast.         
        and_image = cv2.bitwise_and(mask2,(255-mask))
        if np.sum(and_image[:]>0): found = False; #out of the limis of the breast. 
        idx +=1
        #print("found ",found)
        #input("Press Enter to continue...")
    if not found: print("*** Could not get a Normal random ROI after 100 iterations")
    return bbox_random

In [ ]:
def get_normal_BBox (image,bbox):
    #threshold image 
    img = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY)[1]  # ensure binary
#     plot.imshow(img,'gray')#,vmin=0,vmax=255))
#     plot.show()
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
    sizes = stats[:, -1]
    max_label = 1
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]
    img2 = np.zeros(output.shape,dtype=np.uint8)
    img2[output == max_label] = 255
#     plot.imshow(img2,'gray')
#     plot.show()
#     input("Press Enter to continue...")
    contours, hierarchy = cv2.findContours(img2,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    cnt = contours[0]
    aux_im = img2
    x,y,w,h = cv2.boundingRect(cnt)
    cv2.rectangle(aux_im,(x,y),(x+w,y+h),(255,0,0),5)
#     plot.imshow(aux_im)
#     plot.show()
    #input("Press Enter to continue...")
    out_bbox = omidb.mark.BoundingBox(x, y, x+w, y+h)
    
    return out_bbox, img2 # returns bounding box and mask image. 

## copy_case

The copy format will be: 
scanner/ffdm/msxxx/client_episode_series_CC
scanner/roi/msxxx/client_episode_
series_CC

Where xxx is the subtpye

In [ ]:
def copy_case(scanner, subtype,view, client, episode,image,side):
    # create folders
    try:
        os.mkdir(output_path)     
    except OSError as e:
        if (e.errno != 17): 
            print("copy_case: creation of %s failed" % output_path)
            print(e)
    try: 
        os.mkdir(output_path+"/"+str(scanner))        
        os.mkdir(output_path+"/"+str(scanner)+"/ffdm")
        os.mkdir(output_path+"/"+str(scanner)+"/roi")
        os.mkdir(output_path+"/"+str(scanner)+"/normal_roi")
        
        for i in range(8):
            os.mkdir(output_path+"/"+str(scanner)+"/ffdm/st"+"{0:03b}".format(i))
            os.mkdir(output_path+"/"+str(scanner)+"/roi/st"+"{0:03b}".format(i))
            os.mkdir(output_path+"/"+str(scanner)+"/normal_roi/st"+"{0:03b}".format(i))
    
    except OSError as e:
        if (e.errno != 17): 
            print("copy_case: creation of %s failed" % output_path)
            print(e)
    
    filename =client+"_"+episode+"_"+image.id+"_"+view+".png"
    print(filename)
    
    if 'WindowWidth' in image.dcm:
        #print('Image hs windowing')
        image_2d = apply_voi_lut(image.dcm.pixel_array, image.dcm)
    else: image_2d = image.dcm.pixel_array
    # Rescaling grey scale between 0-255
    image_2d_scaled = (np.maximum(image_2d,0) / image_2d.max()) * 255.0

    # Convert to uint
    image_2d_scaled = np.uint8(image_2d_scaled)
    if (side=='R'): image_2d_scaled =cv2.flip(image_2d_scaled, 1)
    dims = image_2d_scaled.shape
    print("dimension",dims)
   
    # get ROI from mask
     # Write the PNG file
    aux_folder = output_path+"/"+str(scanner)+"/roi/st"+subtype+"/"+ filename
    for mark in image.marks:
        bbox_roi = mark.boundingBox
        if (side=='R'): # mirrowing the marks
            aux = bbox_roi.x2 
            bbox_roi.x2 = dims[1]-mark.boundingBox.x1 
            bbox_roi.x1 = dims[1]-aux 
        print("bbox_roi",bbox_roi)
        #print(type(bbox))
        #adding an extra_size around the lesion.
        y1 = np.maximum(bbox_roi.y1-extra_size,0)
        y2 = np.minimum(bbox_roi.y2+extra_size,dims[0])
        x1 = np.maximum(bbox_roi.x1-extra_size,0)
        x2 = np.minimum(bbox_roi.x2+extra_size,dims[1])
        
        image_crop = image_2d_scaled[y1:y2,x1:x2]

        cv2.imwrite(aux_folder,image_crop)

    #     #get normal ROI
    # TODO if tehy have more than one mark put it in the asme loop. 
    aux_folder = output_path+"/"+str(scanner)+"/normal_roi/st"+subtype+"/"+ filename
    for mark in image.marks:
        bbox,mask = get_normal_BBox(image_2d_scaled,bbox_roi)
        #print(bbox)
        # get_random_bbox(bbox)
        bbox_norm = get_random_bbox(bbox, bbox_roi,mask)
        image_crop = image_2d_scaled[bbox_norm.y1:bbox_norm.y2,bbox_norm.x1:bbox_norm.x2]
        cv2.imwrite(aux_folder,image_crop)

    aux_folder = output_path+"/"+str(scanner)+"/ffdm/st"+subtype+"/"+ filename
    print(aux_folder)        
    # Write the PNG file
    image_crop = image_2d_scaled[bbox.y1:bbox.y2,bbox.x1:bbox.x2]
    cv2.imwrite(aux_folder,image_crop)
    
    # write CSV 
    with open(output_path+'/omidb-selection.csv', 'a+', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([client,subtype, episode, image.id, filename, side, scanner, bbox, bbox_roi, extra_size])

        
    
               
    
    

## get_subtype_from_event()

Get sub-type for event surgery


In [ ]:
#todo_ what happends if data is not available!?
def get_subtype_from_event(episode):
    side = 'U'
    subtype = np.zeros(3, dtype=np.int8)
    not_known = False;
    print("EPISODE ->")
    #print(episode_data)
    for key, value in episode_data.items():
        #print ("new key ",key, value)
        if key == "SURGERY":
            #print ("new key ",key, value)       
            for key1, value1 in value.items():
                #print ("new key1",key1, value1)
                if (key1 == 'R' or key1 == 'L'): # here we have subtype info 
                    # for all findings
                    side = key1
                    for key_finding, value_finding in value1.items():                    
                        for key_st, value_st in value_finding.items():
                            if (key_st =='HormoneERStatus'):   
                                #print(key_st, value_st)
                                if value_st == 'RP':
                                    subtype[ER]= True
                                elif value_st == 'RN':
                                    subtype[ER]= False
                                else: print("*** ER Status NA",value_st);not_known = True;
                            if (key_st=='HormonePRStatus'):   
                                if value_st == 'RP':
                                    subtype[PR]= True
                                elif value_st == 'RN':
                                    subtype[PR]= False
                                else: print ("*** PR Status NA",value_st );not_known = True;
                                #print(key_st, value_st)
                            if (key_st=='HER2ReceptorStatus'):   
                                if value_st == 'RP':
                                    subtype[HER2]= True
                                elif value_st == 'RN':
                                    subtype[HER2]= False
                                else: print ("*** HER2 status NA",value_st);not_known = True;
                                #print(key_st, value_st)
#                 else: print("*** Side unknown", key1)
                        
    return subtype, side, not_known
    

## Get all images 
Get all images from HOLOGIC with mass and put it in its corresponding bin (ROI and image).



In [ ]:
# example for accessing nbss data of a given episode
#
overall = stats(0,0,0)
copied_count = stats(0,0,0)
# write CSV 
with open(output_path+'/omidb-selection.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["client","subtype", "episode", "image", "filename","side", "scanner", "bbox", "bbox_roi", "extra_size"])

for client in db:
    print(client.status) # interested in 'M' or CI interval cancer?
    if (client.status==client.status.M):
        overall.M +=1
    if (client.status==client.status.CI): 
        overall.IC +=1
    if (client.status==client.status.N):
        overall.N +=1
    #for all episodes Recurrent high risk? not used.
    nbss_data = db._nbss(client.id) # Access 'raw' NBSS data
    for episode in client.episodes:
        if (debug):
            pass
            #print(episode.type)
            #print(episode.has_malignant_opinions)
            #print(episode.has_benign_opinions)
            #print(episode.events)
            #print(episode.studies)
        # we want only malignant cases
        if episode.has_malignant_opinions: 
            if (episode.studies is not None):
                # recover the episode
                #normally one episode has one study?
                episode_data = nbss_data.get(episode.id, {})
                #print(episode_data)
                # recover the malignacy score for the episode
                subtype, side, not_known = get_subtype_from_event(episode)
                if (side =='U'):
                    print("*** Side U", client.id, episode.id)
                if (not_known):
                    print("subtype unkown")
                else:
                    print ("subtype ", subtype, " side ", side)
                    # we are interested on the surgery event. HAS surgery event?
                    #print(len(episode.studies))
                    for study in episode.studies:
                        for serie in study.series:
                            # check view CC / MLO, laterality  and presentation type
                            # all images
                            for image in serie.images: 
                                if image.dcm_path.is_file():
                                    #print (image.attributes)
                                    if image.marks and image.dcm.PresentationIntentType == 'FOR PRESENTATION' and\
                                    image.dcm[0x0020,0x0062].value ==side :
                                        escaner = image.dcm['Manufacturer'].value
                                        view = image.dcm[0x0018,0x5101].value
                                        #if any (x in escaner for x in system_type):
                                        for x in system_type:
                                            if (x in escaner):
                                                print("-->> Copying case: ",x, " ", \
                                                view, " ", side, subtype, "IDs ", \
                                                      client.id, episode.id, serie.id,image.id)
                                                #print ("IDs ",client.id, episode.id, serie.id,image.id)

                                                #image.plot()
                                                #update stats
                                                if (side=="R"): copied_count.image_R +=1
                                                else: copied_count.image_L +=1
                                                if (view=="MLO" or view=="ML"): copied_count.image_MLO +=1
                                                elif (view=="CC"): copied_count.image_CC +=1
                                                else: print ("*** Warning view not supported: ", view)                            
                                                sep = ''
                                                aux_st = sep.join(map(str, subtype))
                                                copied_count.subtype[int(aux_st,2)] +=1
                                                copied_count.M +=1

                                                copy_case(x, aux_st, view,
                                                         client.id, episode.id,image,side)                                        
print("SUMMARY copied  ", copied_count)
print("SUMMARY overall ", overall)